Reference : https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko

In [1]:
# HDF5 포맷으로 모델을 저장하기 위해 필요
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
# IMDB 데이터셋 준비
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [7]:
# Sequential 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)])
    
    model.compile(optimizer = 'adam',
                  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
    
    return model

# 설정한 모델 생성
model = create_model()

# 모델 구조 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 체크포인트 콜백 사용해서 저장하기

In [8]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다.
          
# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있음

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1112 - accuracy: 0.6996
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 16ms/step - loss: 1.1079 - accuracy: 0.7000 - val_loss: 0.7063 - val_accuracy: 0.7800
Epoch 2/10
17/32 [==============>...............] - ETA: 0s - loss: 0.4383 - accuracy: 0.8750
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4215 - accuracy: 0.8800 - val_loss: 0.5364 - val_accuracy: 0.8300
Epoch 3/10
17/32 [==============>...............] - ETA: 0s - loss: 0.2934 - accuracy: 0.9228
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2857 - accuracy: 0.9260 - val_loss: 0.4683 - val_accuracy: 0.8510
Epoch 4/10
17/32 [==============>...............] - ETA: 0s - loss: 0.2285 - accuracy: 0.9485
Epoch 00004: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 

- 위 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다.
- training_1/cp.ckpt

In [10]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

- 두 모델이 동일한 아키텍처를 공유하기만 한다면 두 모델 간에 가중치를 공유할 수 있음. 따라서 모델을 복원할 때 원래 모델과 동일한 아키텍처로 모델을 만든 다음 가중치를 설정

- 훈련되지 않은 새로운 모델을 만들고, 테스트셋에서 평가 진행

In [12]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('훈련되지 않은 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 2.4149 - accuracy: 0.1100 - 171ms/epoch - 5ms/step
훈련되지 않은 모델의 정확도: 11.00%


In [14]:
# 가중치 불러와서 정확도 비교
model.load_weights(checkpoint_path)

# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('가중치 복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4295 - accuracy: 0.8620 - 67ms/epoch - 2ms/step
가중치 복원된 모델의 정확도: 86.20%


## 체크포인트 콜백 매개변수

In [18]:
# 파일 이름에 에포크 번호를 포함시키기('str.format')
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

# 새로운 모델 객체 생성
model = create_model()

# 'checkpoint_path' 포맷을 사용하여 가중치를 저장
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델 훈련
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data = (test_images, test_labels),
          verbose= 0)


Epoch 00001: saving model to training_2\cp-0001.ckpt

Epoch 00001: saving model to training_2\cp-0001.ckpt

Epoch 00001: saving model to training_2\cp-0001.ckpt

Epoch 00001: saving model to training_2\cp-0001.ckpt

Epoch 00001: saving model to training_2\cp-0001.ckpt

Epoch 00001: saving model to training_2\cp-0001.ckpt

Epoch 00002: saving model to training_2\cp-0002.ckpt

Epoch 00002: saving model to training_2\cp-0002.ckpt

Epoch 00002: saving model to training_2\cp-0002.ckpt

Epoch 00002: saving model to training_2\cp-0002.ckpt

Epoch 00002: saving model to training_2\cp-0002.ckpt

Epoch 00002: saving model to training_2\cp-0002.ckpt

Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00003: saving model t

In [19]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0001.ckpt.data-00000-of-00001',
 'cp-0001.ckpt.index',
 'cp-0002.ckpt.data-00000-of-00001',
 'cp-0002.ckpt.index',
 'cp-0003.ckpt.data-00000-of-00001',
 'cp-0003.ckpt.index',
 'cp-0004.ckpt.data-00000-of-00001',
 'cp-0004.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0006.ckpt.data-00000-of-00001',
 'cp-0006.ckpt.index',
 'cp-0007.ckpt.data-00000-of-00001',
 'cp-0007.ckpt.index',
 'cp-0008.ckpt.data-00000-of-00001',
 'cp-0008.ckpt.index',
 'cp-0009.ckpt.data-00000-of-00001',
 'cp-0009.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0011.ckpt.data-00000-of-00001',
 'cp-0011.ckpt.index',
 'cp-0012.ckpt.data-00000-of-00001',
 'cp-0012.ckpt.index',
 'cp-0013.ckpt.data-00000-of-00001',
 'cp-0013.ckpt.index',
 'cp-0014.ckpt.data-00000-of-00001',
 'cp-0014.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0016.ckpt.data-00000

In [20]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

- 기본 TF 형식은 가장 최근의 5개 체크포인트만 저장됨

In [23]:
# 모델 생성
model = create_model()

# 가중치 불러오기
model.load_weights(latest)

# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose= 2)
print(f'model accuracy with latest weights: {acc*100:.2f}%')

32/32 - 0s - loss: 0.4904 - accuracy: 0.8750 - 160ms/epoch - 5ms/step
model accuracy with latest weights: 87.50%


- 위 코드들은 이진 형식의 훈련된 가중치만 포함하는 체크포인트 형식의 파일 모음에 가중치를 저장한다.

- 체크포인트 형식이 갖는 데이터
  - 모델의 가중치를 포함하는 하나 이상의 샤드
  - 어떤 가중치가 어떤 샤드에 저장되어 있는지 나타내는 인덱스 파일
  
- 단일 머신에서 모델을 훈련하는 경우 접미사가 .data-00000-of-00001인 하나의 샤드를 갖게 됨

## 수동으로 가중치 저장하기

- Model.save_weights 메서드를 사용하여 수동으로 가중치를 저장합니다. 기본적으로 tf.keras.Model.save_weights는 .ckpt 확장자가 있는 TF 체크포인트 형식을 사용합니다. (.h5 확장자를 사용하여 hdf5에 저장하는 내용은 모델 저장 및 직렬화 가이드에서 다룸)

In [24]:
# 가중치 저장
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체 생성
model = create_model()

# 가중치 복원
model.load_weights('./checkpoints/my_checkpoint')

# 모델 평가
loss, acc = model.evaluate(test_images, test_labels, verbose= 2)
print(f'복원된 모델의 정확도: {acc*100:.2f}%')

32/32 - 0s - loss: 0.4904 - accuracy: 0.8750 - 171ms/epoch - 5ms/step
복원된 모델의 정확도: 87.50%


## 전체 모델 저장하기
model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 젖아합니다. 모델을 저장하기 때문에 원본 파이썬 코드가 없어도 사용할 수 있습니다. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 재시작 가능.

전체 모델은 두 가지 다른 파일 형식(SavedModel & HDF5)으로 저장할 수 있습니다. TensorFlow SavedModel 형식은 TF2.x의 기본 파일 형식입니다. 그러나 모델을 HDF5 형식으로 저장할 수 있습니다. 

전체 모델을 저장하는 기능을 통해서 TensorFlow.js로 모델을 이식하는 것이 가능합니다. (사용자 정의 객체 -- 상속으로 만들어진 클래스나 레이어 -- 는 특별한 주의가 필요할 수 있습니다.)



### SavedModel 포맷
해당 포맷은 모델을 직렬화하는 방법 중 하나입니다. 이 형식으로 저장된 모델은 tf.keras.models.load_model 메서드를 호출하여 복원할 수 있으며, Tesorflow Serving과 호환됩니다.

In [25]:
# 새로운 모델 객체를 만들고 훈련합니다.
model = create_model()
model.fit(train_images, train_labels, epochs = 5 )

# SavedModel 로 전체 모델을 저장
!mkdir -p saved_model
model.save('saved_model_my_model')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1292 - accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4123 - accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2793 - accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2096 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1585 - accuracy: 0.9630
INFO:tensorflow:Assets written to: saved_model_my_model\assets


In [38]:
ls saved_model\saved_model_my_model

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 12BB-23DC

 C:\Users\user\Desktop\DataAnalysisStudy\datascienceschool\saved_model\saved_model_my_model 디렉터리

2022-10-05  오전 10:39    <DIR>          .
2022-10-05  오전 10:39    <DIR>          ..
2022-10-05  오전 10:39    <DIR>          assets
2022-10-05  오전 10:39             7,358 keras_metadata.pb
2022-10-05  오전 10:39            76,269 saved_model.pb
2022-10-05  오전 10:39    <DIR>          variables
               2개 파일              83,627 바이트
               4개 디렉터리  44,158,283,776 바이트 남음


assets, keras_metadata.pb, saved_model.pb, variables 확인

In [39]:
# 저장한 경로의 모델 불러오기
new_model = tf.keras.models.load_model('saved_model/saved_model_my_model')

# 모델 구조 확인
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [42]:
# 복원된 모델에 대한 평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'복원된 모델 정확도 : {acc*100:.2f}%')
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4511 - accuracy: 0.8520 - 66ms/epoch - 2ms/step
복원된 모델 정확도 : 85.20%
(1000, 10)


### HDF5 파일 포맷으로 저장
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공한다.

In [43]:
model = create_model()
model.fit(train_images, train_labels, epochs = 5)

# 전체 모델을 HDF5 파일로 저장
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1712 - accuracy: 0.6660
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4329 - accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2902 - accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2009 - accuracy: 0.9580
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1643 - accuracy: 0.9630


In [44]:
# 저장한 파일로부터 모델 불러오기
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조 확인
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 512)               401920    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [45]:
# 복원된 모델의 정확도 확인
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'복원된 모델 정확도 : {acc*100:.2f}%')

32/32 - 0s - loss: 0.4313 - accuracy: 0.8630 - 127ms/epoch - 4ms/step
복원된 모델 정확도 : 86.30%


Keras는 아키텍처를 검사하여 모델을 저장합니다. 이 기술은 모든 내용을 저장합니다.
- 가중치 값
- 모델 구조
- 모델의 훈련 구성 ( compile 메서드로 설정한 내용 )
- 존재하는 옵티마이저와 그 상태

체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(tf.compat.v1.train)을 저장할 수 없습니다. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다. 따라서 옵티마이저의 상태를 잃게 됩니다. 

## 사용자 정의 객체
SavedModel 형식을 사용하는 경우 HDF5 형식에 비해 신경을 안 써도 되는 부분입니다. 둘의 주요 차이점은, HDF5 포맷은 객체 구성을 사용하여 모델 아키텍처를 저장하는 반면, SavedModel 포맷은 실행 그래프를 저장한다는 것입니다. 따라서 SavedModel 은 원본 코드 없이도 서브클래싱된 모델 및 사용자 지정 레이어와 같은 사용자 지정 객체를 저장할 수 있습니다.

사용자 정의 객체를 HDF5로 저장하기 위해서는 .. TF Document 참조...

---
***딥러닝 메모***
- 오버피팅을 해결하기 위한 방법
  - 정칙화 ( Regularization )
    - L1, L2 정칙화, Dropout, Batch normalization 등이 있음.
      - Dropout : 확률적으로 랜덤하게 몇 가지의 뉴럴만 선택하여 전달
    - 모델이 train set의 정답을 맞히지 못하도록 오버피팅을 방해 ( train loss가 증가 ) 하는 역할
    -  train loss는 약간 증가하지만 결과적으로, validation loss나 최종적인 test loss를 감소시키려는 목적
    
  - 정규화 ( normalization )
    - 데이터의 형태를 좀 더 의미있게, 혹은 트레이닝에 적합하게 전처리하는 과정
    - 데이터를 z-score로 바꾸거나 min-max scaler를 사용하여 0과 1 사이의 값으로 조정하는 것들이 해당
    - 모든 피처의 범위 분포를 동이랗게 하여 모델이 풀어야 하는 문제를 좀 더 간단하게 바꾸어 주는 전처리 과정

- ds -> model -> loss 도출
- loss에 optimizer를 통해 w, b ( weight, bias ) 를 업데이트하는 게 안공신경망의 프로세스

- 레이어 구성 순서
  - Convolution > Batch Normalization > Activation > Dropout > Pooling
  - 배치 정규화는 합성곱 레이어 또는 완전연결레이어(Dense) 뒤에 적용하며, ReLU와 같은 활성 함수를 적용하기 전에 적용했을 때 목적에 맞게끔 네트워크를 구성할 수 있음
  - 반면 Dropout은 활성 함수 다음에 적용하는 것이 좋고, 마지막으로 Pooling 연산이 들어가는 것이 추천됨